# ICL Step

In [1]:
import json
from reasoning_functions import *
with open('train_gsm8k_llama2_7b_full_weak.json') as f:
    llama_dat = json.load(f)
with open('train_gsm8k_gemma_2b_full_weak.json') as f:
    gemma_dat = json.load(f)
    #print(d)
with open('train_gsm8k_mistral_7b_full_weak.json') as f:
    mistral_dat = json.load(f)

In [2]:
mistral_dat[2]

{'id': 2,
 'content': 'Question: Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?\nAnswer:',
 'output': 'There are 10 * 0.8 = 8 more purple flowers than yellow flowers.\nSo there are 10 + 8 = 18 purple flowers.\nGreen flowers are 25/100 * (10 + 18) = 5.5 times less numerous than yellow and purple flowers.\nSo there are 10 / 5.5 = 1.81818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818181818'}

In [3]:

llama_icl_examples = ['''Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. 
How many clips did Natalia sell altogether in April and May?\nAnswer: 
Natalia sold 48 x 1/2 = 24 clips in May.\nSo, she sold 48 + 24 = 72 clips altogether in April and May.\n#### 72\n\n''', 
''' Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?\nAnswer:
She earns $12 an hour and she just did 50 minutes of babysitting so she earned 12/60 = $0.2 an hour.\nSo she earned $0.2 x 50 = $10.\n#### 10\n\n''',
'''Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there 
are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. 
How many flowers does Mark have in his garden? \nAnswer: There are 10 x 1.8 = 18 purple flowers.\n\nThere are 10 x 25% = 
2.5 green flowers.\nThere are 10 + 18 + 2.5 = 30 flowers in total.\n#### 30''']

gemma_icl_examples = ['''Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. 
How many clips did Natalia sell altogether in April and May?\nAnswer: In April, Natalia sold 48 clips.\nIn May, 
Natalia sold half as many clips as she did in April, so she sold 48/2=24 clips.\nIn April and May, Natalia sold 48+24=72 clips.\n#### 72''',
 '''Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?\nAnswer:
 12 * 50 / 60 = 10\nShe earned 10 dollars.\n#### 10 ''', '''Question: Mark has a garden with flowers. He planted plants 
 of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as 
 there are yellow and purple flowers. How many flowers does Mark have in his garden?\nAnswer: There are 10 x 80/100 = 8 
 purple flowers.\nThere are 25/100 x 10 = 2.5 green flowers.\nSo, there are 10 + 8 + 2.5 = 12.5 green flowers.\nTherefore, Mark has 
 12.5 x 100/25 = 50 flowers in his garden.\n#### 50 ''' ]

mistral_icl_examples = ['''Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. 
How many clips did Natalia sell altogether in April and May?\nAnswer: In May, Natalia sold 48/2 = 24 clips.\nAltogether, 
Natalia sold 48 + 24 = 72 clips.\n#### 72''', '''Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 
50 minutes of babysitting. How much did she earn?\nAnswer: She was paid for 50 minutes of babysitting.\nSo she was paid for 50/60 = 0.833 
of an hour.\nSo she earned 12 * 0.833 = $9.99.\n#### 9.99''', '''Question: Mark has a garden with flowers. He planted plants of 
three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. 
There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?\nAnswer:
There are 10 * 0.8 = 8 more purple flowers than yellow flowers.\nSo there are 10 + 8 = 18 purple flowers.\nGreen flowers 
are 25/100 * (10 + 18) = 5.5 times less numerous than yellow and purple flowers.\nSo there are 10 / 5.5 = 1.81818''']

# getting the training examples, I use the first 200 samples exluding the icl examples 
training_questions = [llama_dat[i]['content'] for i in range(3, 203)]
llama_ans = [llama_dat[i]['output'] for i in range(3, 203)]
gemma_ans = [gemma_dat[i]['output'] for i in range(3, 203)]
mistral_ans = [mistral_dat[i]['output'] for i in range(3, 203)]

In [7]:
##Construct ICL Data
improvement_model = 'gpt-4o-mini-2024-07-18'
ICL_llama_ans = []
ICL_gemma_ans = []
ICL_mistral_ans = []
for question in training_questions:
    system_prompt = ''''''
    llama_ICL_prompt = ''.join(llama_icl_examples)
    gemma_ICL_prompt = '\n\n'.join(gemma_icl_examples)
    mistral_ICL_prompt = '\n\n'.join(mistral_icl_examples)
    llama_prompt = llama_ICL_prompt+question
    gemma_prompt = gemma_ICL_prompt+question
    mistral_prompt = mistral_ICL_prompt+question
    llama_prompt = FormatInput(system_prompt, gemma_prompt, improvement_model)
    gemma_prompt = FormatInput(system_prompt, gemma_prompt, improvement_model)
    mistral_prompt = FormatInput(system_prompt, mistral_prompt, improvement_model)
    ICL_llama_ans.append(QueryModel(llama_prompt, improvement_model, api='OPENAI'))
    ICL_gemma_ans.append(QueryModel(gemma_prompt, improvement_model, api='OPENAI'))
    ICL_mistral_ans.append(QueryModel(mistral_prompt, improvement_model, api='OPENAI'))

In [8]:
# here we save the improved icl examples as a jsonl file
system_prompt = ''''''
SaveJSONL(system_prompt, training_questions, ICL_llama_ans, 'gsm8k_llama2_ICL_mini.jsonl')
SaveJSONL(system_prompt, training_questions, llama_ans, 'gsm8k_llama2_weak.jsonl')
SaveJSONL(system_prompt, training_questions, ICL_gemma_ans, 'gsm8k_gemma_ICL_mini.jsonl')
SaveJSONL(system_prompt, training_questions, gemma_ans, 'gsm8k_gemma_weak.jsonl')
SaveJSONL(system_prompt, training_questions, ICL_mistral_ans, 'gsm8k_mistral_ICL_mini.jsonl')
SaveJSONL(system_prompt, training_questions, mistral_ans, 'gsm8k_mistral_weak.jsonl')

# Finetuning

In [4]:
improvement_model = 'gpt-4o-mini-2024-07-18'
frac_train = 1 #Do not modify this. Validation data will not be used anyways...

In [4]:
CheckTokens(f'gsm8k_llama2_weak.jsonl')
CheckTokens(f'gsm8k_llama2_ICL_mini.jsonl')
CheckTokens(f'gsm8k_gemma_weak.jsonl')
CheckTokens(f'gsm8k_gemma_ICL_mini.jsonl')
CheckTokens(f'gsm8k_mistral_weak.jsonl')
CheckTokens(f'gsm8k_mistral_ICL_mini.jsonl')

Processing file: gsm8k_llama2_weak.jsonl

#### Distribution of total tokens:
min / max: 67, 562
mean / median: 161.81, 148.5
p5 / p95: 103.9, 222.0

#### Distribution of assistant tokens:
min / max: 17, 494
mean / median: 83.35, 69.0
p5 / p95: 43.0, 125.0
**************************************************
Processing file: gsm8k_llama2_ICL_mini.jsonl

#### Distribution of total tokens:
min / max: 97, 595
mean / median: 291.73, 275.5
p5 / p95: 165.60000000000002, 446.5

#### Distribution of assistant tokens:
min / max: 50, 513
mean / median: 213.27, 196.0
p5 / p95: 105.9, 359.1
**************************************************
Processing file: gsm8k_gemma_weak.jsonl

#### Distribution of total tokens:
min / max: 62, 628
mean / median: 174.735, 150.5
p5 / p95: 96.0, 241.29999999999998

#### Distribution of assistant tokens:
min / max: 10, 512
mean / median: 96.275, 70.5
p5 / p95: 34.900000000000006, 154.29999999999998
**************************************************
Processing file: gs

In [12]:
weak_fine_tune_job = FineTune('gsm8k_llama2_weak.jsonl')
llama_weak_GPT_mini = GetFineTunedModelName(weak_fine_tune_job)

In [13]:
llama_ICL_fine_tune_job = FineTune('gsm8k_llama2_ICL_mini.jsonl')
llama_ICL_gpt = GetFineTunedModelName(llama_ICL_fine_tune_job)

In [14]:
gemma_weak_fine_tune_job = FineTune('gsm8k_gemma_weak.jsonl')
gemma_weak_GPT_mini = GetFineTunedModelName(gemma_weak_fine_tune_job)
print(gemma_weak_GPT)

In [15]:
gemma_ICL_fine_tune_job = FineTune('gsm8k_gemma_ICL_mini.jsonl')
gemma_ICL_gpt_mini = GetFineTunedModelName(gemma_ICL_fine_tune_job)
print(gemma_ICL_GPT)

In [16]:
mistral_weak_fine_tune_job = FineTune('gsm8k_mistral_weak.jsonl')
mistral_weak_GPT = GetFineTunedModelName(mistral_weak_fine_tune_job)
print(mistral_weak_GPT)

In [17]:
mistral_ICL_fine_tune_job = FineTune('gsm8k_mistral_ICL_mini.jsonl')
mistral_ICL_GPT_mini = GetFineTunedModelName(mistral_ICL_fine_tune_job)
print(mistral_ICL_GPT)

In [11]:
print(llama_weak_GPT)

ft:gpt-3.5-turbo-0125:university-of-michigan::9u7BAH6O


In [18]:
llama_ICL_id = 
llama_weak_id = 
gemma_weak_id  = 
gemma_ICL_id = 
mistral_weak_id = 
mistral_ICL_id = 

# Test Responses

In [19]:
with open('gsm8k.jsonl') as f:
    test_data = [json.loads(line) for line in f]
small_test = [test_data[i]['question'] for i in range(100)]
small_test_key = [test_data[i]['solution'] for i in range(100)]

In [21]:
##Construct test data
llama_ICL_gpt_test_ans = []
llama_weak_gpt_test_ans = []
gemma_weak_gpt_test_ans = []
gemma_ICL_gpt_test_ans = []
mistral_weak_gpt_test_ans = []
mistral_ICL_gpt_test_ans = []
baseline_ans = []
for i, question in enumerate(small_test):
    system_prompt = ''''''
    user_prompt = question
    key = small_test_key[i]
    prompt = FormatInput(system_prompt, user_prompt,'gpt-4o-mini-2024-07-18')
    llama_ICL_gpt_test_ans.append(QueryModel(prompt, llama_ICL_id, api='OPENAI'))
    llama_weak_gpt_test_ans.append(QueryModel(prompt, llama_weak_id, api='OPENAI'))
    gemma_weak_gpt_test_ans.append(QueryModel(prompt,gemma_weak_id,api = 'OPENAI'))
    gemma_ICL_gpt_test_ans.append(QueryModel(prompt,gemma_ICL_id,api = 'OPENAI'))
    mistral_weak_gpt_test_ans.append(QueryModel(prompt,mistral_weak_id,api = 'OPENAI'))
    mistral_ICL_gpt_test_ans.append(QueryModel(prompt,mistral_ICL_id,api = 'OPENAI'))
    baseline_ans.append(QueryModel(prompt, 'gpt-4o-mini-2024-07-18',api='OPENAI'))
SaveJSONL(system_prompt, small_test, llama_ICL_gpt_test_ans, 'gsm8k_llama2_mini_ICL_test.jsonl')
SaveJSONL(system_prompt, small_test, llama_weak_gpt_test_ans, 'gsm8k_llama2_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, baseline_ans, 'gsm8k_baseline_mini_test.jsonl')
SaveJSONL(system_prompt, small_test, mistral_weak_gpt_test_ans, 'gsm8k_mistral_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, mistral_ICL_gpt_test_ans, 'gsm8k_mistral_mini_ICL_test.jsonl')
SaveJSONL(system_prompt, small_test, gemma_weak_gpt_test_ans, 'gsm8k_gemma_mini_weak_test.jsonl')
SaveJSONL(system_prompt, small_test, gemma_ICL_gpt_test_ans, 'gsm8k_gemma_mini_ICL_test.jsonl')

# Eval

In [22]:
with open('gsm8k_llama2_mini_ICL_test.jsonl') as f:
    llama_ICL_test_resp = [json.loads(line) for line in f]
with open('gsm8k_llama2_mini_weak_test.jsonl') as f:
    llama_weak_test_resp = [json.loads(line) for line in f]
with open('gsm8k_baseline_mini_test.jsonl') as f:
    baseline_resp = [json.loads(line) for line in f]
with open('gsm8k_mistral_mini_weak_test.jsonl') as f:
    mistral_weak_test_resp = [json.loads(line) for line in f]
with open('gsm8k_mistral_mini_ICL_test.jsonl') as f:
    mistral_ICL_test_resp = [json.loads(line) for line in f]
with open('gsm8k_gemma_mini_weak_test.jsonl') as f:
    gemma_weak_test_resp = [json.loads(line) for line in f]
with open('gsm8k_gemma_mini_ICL_test.jsonl') as f:
    gemma_ICL_test_resp = [json.loads(line) for line in f]

In [23]:

llama_ICL_scores, llama_weak_scores, baseline = [], [], []
mistral_ICL_scores, mistral_weak_scores, gemma_ICL_scores, gemma_weak_scores = [], [], [], []
gemma_weak_scores = []
llama_rep_scores = []
eval_model = 'gpt-4o'
for i, testq in enumerate(small_test):
    eval_sys = GetEvalSystemPrompt()
    key = small_test_key[i]
    eval_prompt_baseline = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, baseline_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_llama_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, llama_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_llama_rep = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, llama_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_llama_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, llama_ICL_test_resp[i]['messages'][2]['content']), model=eval_model)
    try: #sometimes the gemma trained model fails to provide an answer
        eval_prompt_gemma_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, gemma_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
        gemma_weak_scores.append(GPTEval(eval_prompt_gemma_weak, model=eval_model))
    except: # if answer is blank score 0
        print(i)
        gemma_weak_scores.append(0)
    eval_prompt_gemma_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, gemma_ICL_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_mistral_weak = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, mistral_weak_test_resp[i]['messages'][2]['content']), model = eval_model)
    eval_prompt_mistral_ICL = FormatInput(eval_sys, GetEvalUserPrompt(testq, key, mistral_ICL_test_resp[i]['messages'][2]['content']), model = eval_model)
    llama_ICL_scores.append(GPTEval(eval_prompt_llama_ICL, model=eval_model))
    llama_weak_scores.append(GPTEval(eval_prompt_llama_weak, model=eval_model))
    baseline.append(GPTEval(eval_prompt_baseline, model=eval_model))
    mistral_ICL_scores.append(GPTEval(eval_prompt_mistral_ICL, model=eval_model))
    mistral_weak_scores.append(GPTEval(eval_prompt_mistral_weak, model=eval_model))
    gemma_ICL_scores.append(GPTEval(eval_prompt_gemma_ICL, model=eval_model))
    gemma_weak_scores.append(GPTEval(eval_prompt_gemma_weak, model=eval_model))

In [24]:
dictionary = {'baseline':baseline, 'lweak': llama_weak_scores, 'lICL': llama_ICL_scores, 'gweak': gemma_weak_scores, 'gICL:': gemma_ICL_scores, 'mweak': mistral_weak_scores, 'mICL': mistral_ICL_scores}
np.save('gsm8k_scores_mini.npy', dictionary) 

In [25]:
print('baseline:'+str(sum(baseline)/100)+' +/- '+str(2*np.std(baseline)/10))
print('lweak:'+str(sum(llama_weak_scores)/100)+' +/- '+str(2*np.std(llama_weak_scores)/10))
print('lICL:'+str(sum(llama_ICL_scores)/100)+' +/- '+str(2*np.std(llama_ICL_scores)/10))
print('gweak:'+str(sum(gemma_weak_scores)/100)+' +/- '+str(2*np.std(gemma_weak_scores)/10))
print('gICL:'+str(sum(gemma_ICL_scores)/100)+' +/- '+str(2*np.std(gemma_ICL_scores)/10))
print('mweak:'+str(sum(mistral_weak_scores)/100)+' +/- '+str(2*np.std(mistral_weak_scores)/10))
print('mICL:'+str(sum(mistral_ICL_scores)/100)+' +/- '+str(2*np.std(mistral_ICL_scores)/10))

baseline:[0.9135] +/- 0.05540496367655159
lweak:[0.5195] +/- 0.09982880345872126
lICL:[0.9065] +/- 0.05726525997496214
gweak:[0.77460526] +/- 0.09660641280911364
gICL:[0.92] +/- 0.054258639865002144
mweak:[0.7265] +/- 0.08733218192625214
mICL:[0.9295] +/- 0.05101166533254918
